## Test data.noaa.gov bbox, start, stop filters

In [1]:
from owslib.csw import CatalogueServiceWeb
from owslib.fes import SortBy, SortProperty
from owslib import fes
import datetime as dt

In [2]:
#csw = CatalogueServiceWeb('http://www.ngdc.noaa.gov/geoportal/csw',timeout=60) # NGDC Geoportal
csw = CatalogueServiceWeb('http://data.noaa.gov/csw',timeout=60)

In [3]:
csw.get_operation_by_name('GetRecords').constraints

[Constraint: SupportedISOQueryables - ['apiso:DistanceValue', 'apiso:Abstract', 'apiso:RevisionDate', 'apiso:Subject', 'apiso:KeywordType', 'apiso:Title', 'apiso:CRS', 'apiso:PublicationDate', 'apiso:Type', 'apiso:AlternateTitle', 'apiso:BoundingBox', 'apiso:AnyText', 'apiso:ParentIdentifier', 'apiso:Modified', 'apiso:Operation', 'apiso:Format', 'apiso:TempExtent_end', 'apiso:DistanceUOM', 'apiso:OrganisationName', 'apiso:ServiceType', 'apiso:TempExtent_begin', 'apiso:ResourceLanguage', 'apiso:ServiceTypeVersion', 'apiso:OperatesOn', 'apiso:Denominator', 'apiso:HasSecurityConstraints', 'apiso:OperatesOnIdentifier', 'apiso:GeographicDescriptionCode', 'apiso:Language', 'apiso:Identifier', 'apiso:OperatesOnName', 'apiso:TopicCategory', 'apiso:CreationDate', 'apiso:CouplingType'],
 Constraint: AdditionalQueryables - ['apiso:Lineage', 'apiso:Classification', 'apiso:Creator', 'apiso:Relation', 'apiso:OtherConstraints', 'apiso:SpecificationTitle', 'apiso:ResponsiblePartyRole', 'apiso:Specific

In [4]:
# adjust to match MaxRecordDefault of CSW, if would be cleaner if we pick this up Capabilities XML
# this issue will allow for this: https://github.com/geopython/OWSLib/issues/211
pagesize = 10
sort_property = 'dc:title'  # a supported queryable of the CSW
sort_order = 'ASC'  # should be 'ASC' or 'DESC'

In [5]:
sortby = SortBy([SortProperty(sort_property, sort_order)])
foo=sortby.properties

In [6]:
# hopefully something like this will be implemented in fes soon
def dateRange(start_date='1900-01-01',stop_date='2100-01-01',constraint='overlaps'):
    if constraint == 'overlaps':
        start = fes.PropertyIsLessThanOrEqualTo(propertyname='apiso:TempExtent_begin', literal=stop_date)
        stop = fes.PropertyIsGreaterThanOrEqualTo(propertyname='apiso:TempExtent_end', literal=start_date)
    elif constraint == 'within':
        start = fes.PropertyIsGreaterThanOrEqualTo(propertyname='apiso:TempExtent_begin', literal=start_date)
        stop = fes.PropertyIsLessThanOrEqualTo(propertyname='apiso:TempExtent_end', literal=stop_date)
    return start,stop

In [7]:
val = 'salinity'
box=[-72.0, 41.0, -69.0, 43.0]   # gulf of maine

In [8]:
# specific specific times (UTC) ...

# hurricane sandy
jd_start = dt.datetime(2012,10,26)
jd_stop = dt.datetime(2012,11,2)

# 2014 feb 10-15 storm
jd_start = dt.datetime(2014,2,10)
jd_stop = dt.datetime(2014,2,15)

# 2014 recent
jd_start = dt.datetime(2014,3,8)
jd_stop = dt.datetime(2014,3,11)

# 2014 recent
jd_start = dt.datetime(1988,1,1)
jd_stop = dt.datetime(1988,3,1)

# 2011 
#jd_start = dt.datetime(2013,4,20)
#jd_stop = dt.datetime(2013,4,24)

# ... or relative to now
#jd_now = dt.datetime.utcnow()
#jd_start = jd_now - dt.timedelta(days=3)
#jd_stop = jd_now + dt.timedelta(days=3)

start_date = jd_start.strftime('%Y-%m-%d %H:00')
stop_date  = jd_stop.strftime('%Y-%m-%d %H:00')

jd_start = dt.datetime.strptime(start_date,'%Y-%m-%d %H:%M')
jd_stop = dt.datetime.strptime(stop_date,'%Y-%m-%d %H:%M')

print start_date,'to',stop_date

1988-01-01 00:00 to 1988-03-01 00:00


In [9]:
start,stop = dateRange(start_date,stop_date)
filter1 = fes.PropertyIsLike(propertyname='apiso:AnyText',literal=('*%s*' % val),
                        escapeChar='\\',wildCard='*',singleChar='?')
bbox = fes.BBox(box,crs='urn:ogc:def:crs:OGC:1.3:CRS84')
#filter_list = [fes.And([ bbox, filter1, start,stop]) ]


In [10]:
filter_list = [fes.And([ bbox, filter1]) ]
csw.getrecords2(constraints=filter_list)
csw.results['matches']

2288

In [11]:
filter_list = [fes.And([ bbox, filter1, start,stop]) ]
csw.getrecords2(constraints=filter_list)
csw.results['matches']

0

In [12]:
filter_list = [filter1]
csw.getrecords2(constraints=filter_list)
csw.results['matches']

12110

In [13]:
filter_list = [fes.And([ bbox, filter1]) ]
startposition = 0
maxrecords = 20
while True:
    print 'getting records %d to %d' % (startposition, startposition+pagesize)
    csw.getrecords2(constraints=filter_list,
                    startposition=startposition, maxrecords=pagesize, sortby=sortby)
#    print csw.request
    for rec,item in csw.records.iteritems():
        print item.title
    if csw.results['nextrecord'] == 0:
        break
    startposition += pagesize
    if startposition >= maxrecords:
        break

getting records 0 to 10
1990, 1991, 1992 and 1995 CRETM/LMER Zooplankton Data Sets (NODC Accession 0000029)
AFSC/RACE/GAP:  RACEBASE Database
AIR PRESSURE and Other Data from MULTIPLE SHIPS and Other Platforms From NE Atlantic (limit-40 W) from 19580101 to 19740228 (NODC Accession 7400501)
AIR PRESSURE and Other Data from UNKNOWN From North Pacific Ocean from 19750101 to 19871231 (NODC Accession 8800174)
AIR PRESSURE and Other Data from UNKNOWN From World-Wide Distribution from 19870801 to 19870831 (NODC Accession 8700313)
AIR PRESSURE and Other Data from UNKNOWN From World-Wide Distribution from 19880101 to 19880131 (NODC Accession 8800056)
AIR PRESSURE and Other Data from UNKNOWN From World-Wide Distribution from 19880201 to 19880227 (NODC Accession 8800051)
AIR PRESSURE and Other Data from UNKNOWN From World-Wide Distribution from 19880701 to 19880731 (NODC Accession 8800227)
AIR PRESSURE and Other Data from UNKNOWN From World-Wide Distribution from 19890101 to 19890131 (NODC Access